# LLM to Motion

OpenAI Documentation: https://platform.openai.com/docs/quickstart?context=python

Find your keys here: https://platform.openai.com/api-keys

Keep an eye on your credit usage: https://platform.openai.com/usage

Following this guide: https://wandb.ai/onlineinference/gpt-python/reports/Setting-Up-GPT-4-In-Python-Using-the-OpenAI-API--VmlldzozODI1MjY4

## Set your GPT Key to your ENV

Uncomment this line to check your environment variables. That should be set up in .zshrc or .bashrc

In [16]:
# %env

In [17]:
%pwd

'/Users/liamroy/Documents/Studies/Monash_31194990/PHD/Studies/Study_04/LLM_vocabulary/scripts'

## Imports

In [18]:
import os
from openai import OpenAI
client = OpenAI()

import re

import random
import time
random.seed(time.time())

from openpyxl import load_workbook

## Building The Prompt

In [19]:
# gpt_assistant_prompt = "You are simulating human feedback for training a robot expression model. Your responses should have variability to simulate human interpretation. Different humans often interpret a robot's pose differently in similar scenarios, so stochasticity is encouraged." 
gpt_assistant_prompt = "You are an expert roboticist and understand how to design communicative expressions for human-robot interaction."

robot_morphology = "dog-shaped quadruped"

deployment_context = f"Consider a scenario where you are collaborating with a {robot_morphology} robot to locate and pick strawberries in a strawberry patch."

state_01 = "S01: [Waiting for Input, The robot is in standby mode waiting for a command from the user]"
state_02 = "S02: [Analyzing Object, The robot is analyzing a target object in front of it on the ground]"
state_03 = "S03: [Found Object, The robot has found a target object in front of it on the ground]"
state_04 = "S04: [Needs Help, The robot is experiencing an error and needs help from the user]"
state_05 = "S05: [Confused, The robot is confused and unsure what to do]"
state_06 = "S06: [Unsure, It is unclear as the robot does not appear to be in any of the described states.]"

parameter_01 = "P01: [Body Tilt]"
parameter_01_value_strings = ["tilts its torso to the left",
                              "", # Neutral, so no sentence.
                              "tilts its torso to the right"]

parameter_02 = "P02: [Body Lean]"
parameter_02_value_strings = ["leans its torso backwards",
                              "", # Neutral, so no sentence.
                              "leans its torso forward"]

parameter_03 = "P03: [Body Turn]"
parameter_03_value_strings = ["turns its torso to the left",
                              "", # Neutral, so no sentence.
                              "turns its torso to the right"]

parameter_04 = "P04: [Body Height]"
parameter_04_value_strings = ["lowers its body to the ground",
                              "", # Neutral, so no sentence.
                              "raises its torso as high as it can"]

parameter_05 = "P05: [Body Direction]"
parameter_05_value_strings = ["",  # NAN or neutral, so no sentence.
                              "The robot faces the user in the scene.",
                              "The robot faces a nearby strawberry in the scene."]

parameter_06 = "P06: [Pose Duration]"
parameter_06_value_strings = ["The robot holds this pose for a short duration of 1 second.",
                              "", # Neutral, so no sentence.
                              "The robot holds this pose for a long duration of 8 seconds."]

parameter_07 = "P07: [Motion Velocity]"
parameter_07_value_strings = ["The robot moves slowly to achieve this pose.",
                              "",  # Neutral speed, so no sentence.
                              "The robot moves quickly to achieve this pose."]

parameter_08 = "P08: [Motion Smoothness]"
parameter_08_value_strings = ["",  # NAN, so no sentence.
                              "The robot's motion is smooth without any disturbances.",
                              "The robot's motion is unsmooth and shaky."]

In [20]:
# Helper Functions

# Function to randomly omit a parameter with a XX% chance
def maybe_include(omission_prob, param_string):
        return param_string if random.random() > omission_prob else ""

### Prompt Generating Function

In [21]:
def generate_gpt_user_prompt(parameter_values, omission_probability):

        # This is done to randomize the order in which the states are presented to GPT, to eliminate any bias for states presented first
        state_list = [state_01, state_02, state_03, state_04, state_05]
        random.shuffle(state_list)
        

        # Generate strings for each parameter and randomly omit with 20% probability
        P01_string = maybe_include(omission_probability, parameter_01_value_strings[0] if parameter_values[0] == "left" else parameter_01_value_strings[2] if parameter_values[0] == "right" else "")
        P02_string = maybe_include(omission_probability, parameter_02_value_strings[0] if parameter_values[1] == "backward" else parameter_02_value_strings[2] if parameter_values[1] == "forward" else "")
        P03_string = maybe_include(omission_probability, parameter_03_value_strings[0] if parameter_values[2] == "left" else parameter_03_value_strings[2] if parameter_values[2] == "right" else "")
        P04_string = maybe_include(omission_probability, parameter_04_value_strings[0] if parameter_values[3] == "low" else parameter_04_value_strings[2] if parameter_values[3] == "high" else "")
        P05_string = maybe_include(omission_probability, parameter_05_value_strings[1] if parameter_values[4] == "user" else parameter_05_value_strings[2] if parameter_values[4] == "object" else "")
        P06_string = maybe_include(omission_probability, parameter_06_value_strings[0] if parameter_values[5] == "short" else parameter_06_value_strings[2] if parameter_values[5] == "long" else "")
        P07_string = maybe_include(omission_probability, parameter_07_value_strings[0] if parameter_values[6] == "slow" else parameter_07_value_strings[2] if parameter_values[6] == "fast" else "")
        P08_string = maybe_include(omission_probability, parameter_08_value_strings[1] if parameter_values[7] == "smooth" else parameter_08_value_strings[2] if parameter_values[7] == "shaky" else "")

        # Create a list of pose-related strings (P01 to P04)
        pose_strings = [P01_string, P02_string, P03_string, P04_string]

        # Filter out empty strings and format the sentence with commas and 'and' before the last item
        pose_strings = [pose for pose in pose_strings if pose]  # Remove empty strings
        if len(pose_strings) > 1:
                body_movement_sentence = ", ".join(pose_strings[:-1]) + ", and " + pose_strings[-1] + "."
        elif pose_strings:
                body_movement_sentence = pose_strings[0] + "."
        else:
                body_movement_sentence = ""

        # Add "The robot" at the beginning only if there's any body movement
        if body_movement_sentence:
                body_movement_sentence = "The robot " + body_movement_sentence

        # Create a coherent paragraph by joining all parts
        sentence_parts = [
                P05_string,  # Direction is always first
                body_movement_sentence,  # Fluid body movement sentence
                P08_string,  # Motion smoothness
                P07_string,  # Motion speed
                P06_string,  # Pose duration
        ]

        # Filter out empty strings and join them into a readable paragraph
        expression_string = " ".join([part for part in sentence_parts if part])

        # Output the final paragraph
        # print(expression_string)

        return f'''
{gpt_assistant_prompt}

{deployment_context}

In this scenario, the robot can be in one of 5 possible states. The data in this list is in the format: "State Number: [State Name, State Description]"

{state_list[0]}

{state_list[1]}

{state_list[2]}

{state_list[3]}

{state_list[4]}

{state_06}

—————————

Below is a description of the robot using its body pose to express its internal state:

{expression_string}

—————————

Your task:

Please estimate what state you think the robot is in based on this description. If none of the states seem to match the description, select 'Unsure'.

Your response must be a single line in the exact format shown below (see example and reference):

[State_Number, State_Name]

Reference: 
State_Number = number of the selected robot state (e.g. S01)
State_Name = name of the selected robot state (e.g. Analyzing Object)
'''

### Test Prompt

In [22]:
test_parameter_values = ["left", 
                         "forward",	
                         "neutral", 
                         "neutral", 
                         "object", 
                         "long", 
                         "medium", 
                         "smooth"]

test_omission_probability = 0.2

test_prompt = generate_gpt_user_prompt(test_parameter_values, test_omission_probability)

print(test_prompt)


You are an expert roboticist and understand how to design communicative expressions for human-robot interaction.

Consider a scenario where you are collaborating with a dog-shaped quadruped robot to locate and pick strawberries in a strawberry patch.

In this scenario, the robot can be in one of 5 possible states. The data in this list is in the format: "State Number: [State Name, State Description]"

S04: [Needs Help, The robot is experiencing an error and needs help from the user]

S02: [Analyzing Object, The robot is analyzing a target object in front of it on the ground]

S01: [Waiting for Input, The robot is in standby mode waiting for a command from the user]

S03: [Found Object, The robot has found a target object in front of it on the ground]

S05: [Confused, The robot is confused and unsure what to do]

S06: [Unsure, It is unclear as the robot does not appear to be in any of the described states.]

—————————

Below is a description of the robot using its body pose to express 

### Generate Your Prompt

## Now build the Request

**Read the request documentation to tune your model for your application**

**Request Documentation**: https://platform.openai.com/docs/api-reference/completions/create

**Models**: The model you want to use (i.e. "gpt-4-turbo", "gpt-4", "gpt-4o", "gpt-4o-mini" or "gpt-3.5-turbo")

**Message**: The message being sent (the prompt)

**Temperature**: Number between 0 and 1, higher numbers mean more random, more creative and make results less predictable. This controls the randomness of the responses. A higher temperature (closer to 1.0) will make the model more creative and diverse in its outputs, while a lower temperature (closer to 0.0) will make it more deterministic. Suggestion: Set the temperature to a moderate value, around 0.4–0.6. This will give you some randomness, mimicking the variability in human input, but still maintain enough accuracy to ensure that the responses are meaningful.

**Frequency_penalty**: Number between -2 and 2, where higher numbers penalize new tokens based on their frequency to that point in the response. The higher the number, the lower the probability of repetition. This parameter penalizes repeated tokens in the output, making responses more diverse and creative. Suggestion: Use a low frequency penalty (e.g., 0.0–0.2) to reduce redundancy but still allow the model to use relevant tokens multiple times where necessary.

**Top-p (Nucleus Sampling)**: This parameter determines the probability mass considered for each token in the output. A lower top-p (e.g., 0.7) will cause the model to sample from only the more likely tokens, while a top-p of 1.0 will consider all possible outcomes. Suggestion: Set the top_p to 0.8–0.9 to introduce some controlled randomness while still keeping the majority of the likelihood in more probable tokens, ensuring coherent responses. ~~~~~  Higher top_p values (closer to 1): This will include a broader range of possible tokens, increasing diversity and randomness, making the behaviour more stochastic (i.e., less predictable). A value of top_p = 1 means the model can sample from the full probability distribution, leading to more creative and unpredictable results.
Lower top_p values (closer to 0): This restricts the sampling to a smaller subset of tokens (those with the highest probability), leading to more deterministic and conservative outputs, i.e., less stochastic behavior. For example, setting top_p = 0.1 would mean the model will only consider the most probable 10% of tokens.

### SETUP CELL BELOW


In [23]:
iteration_quantity = 20
gpt_model = "gpt-4o"                    # gpt-3.5-turbo     | Use this for dev/testing
                                        # gpt-4 / gpt-4o    | Use this when deployed, more expensive
                                        # gpt-4o-mini       | Lightweight, less expensive than gpt4o

attempt_ID = '04'

temperature_coefficient = 1.0           # Moderately stochastic @ 0.6 to 1.0
frequency_penalty_coefficient = 1.0     # Lightly penalize repetition @ 0.2
top_p_coefficient=1.0                   # Nucleus sampling for controlled randomness @ 0.85 to 0.6

omission_probability = 0.35

### Multi-Iteration

**Cost Per Run: ~0.60$**

This cell is running the following...

* Iterates through all conditions/states
* Loads that condition/state parameter combination from workbook
* Tests the given combination against the LLM framework
* Logs the results in a new sheet in workbook

In [24]:
row_counter = 1

study_condition = ['LLM', 'HUM', 'RAN']
real_state = ['WFI', 'AO', 'FO', 'NH', 'C']

for condition in study_condition:
    for state in real_state:
        
        row_counter += 1 
        expression_ID = condition + "_" + state

        ## LOAD THE DATA FOR SPECIFIC CONDITION

        workbook_path = "./../data/proxy_validation/proxy_validation.xlsx"
        response_book = load_workbook(workbook_path, data_only=True)
        load_sheet = response_book['HUMAN_ACCURACY']

        # Initialize an empty list for concatenated values
        parameter_list = []     

        # Iterate through the cells Q2 to X2 (columns 17 to 24)
        for col in range(17, 25):  # openpyxl uses 1-based indexing for rows and columns
            cell_value = load_sheet.cell(row=row_counter, column=col).value
            if cell_value is not None:
                parameter_list.append(str(cell_value))  # Convert each cell value to string and append to the list

        
        ## TEST AND LOG THE DATA FOR SPECIFIC CONDITION
        sheet_name = expression_ID + '_' + attempt_ID 

        # Enter the data in spreadsheet format
        workbook_path = "./../data/proxy_validation/proxy_validation.xlsx"
        response_book = load_workbook(workbook_path)

        try: # Try to open existing sheet
            response_sheet = response_book[sheet_name]
        except KeyError:  # If ot doesn't exist. create it
            response_sheet = response_book.create_sheet(title=sheet_name)
        response_sheet["A1"] = "model"
        response_sheet["B1"] = "study cond"
        response_sheet["C1"] = "real state"
        response_sheet["D1"] = "iteration"
        response_sheet["E1"] = "state number"
        response_sheet["F1"] = "state name"
        response_sheet["G1"] = "justification"
        response_sheet["H1"] = "P1 Tilt"
        response_sheet["I1"] = "P2 Lean"
        response_sheet["J1"] = "P3 Turn"
        response_sheet["K1"] = "P4 Body Height"
        response_sheet["L1"] = "P5 Direction"
        response_sheet["M1"] = "P6 Duration"
        response_sheet["N1"] = "P7 Velocity"
        response_sheet["O1"] = "P8 Smoothness"

        response_book.save(workbook_path)

        print(f'\n************************************')
        print(f'********* expression ID: {expression_ID}')
        print(f'************************************\n')

        error_counter = 0

        for iteration in range(0, iteration_quantity):
            print(f'~~~~~~~~~~~~ Iteration {iteration:02d}')

            # adding to excel
            response_sheet["A"+str(iteration+2)] = gpt_model
            response_sheet["B"+str(iteration+2)] = condition
            response_sheet["C"+str(iteration+2)] = state
            response_sheet["D"+str(iteration+2)] = iteration

            
            # calling GPT client
            completion = client.chat.completions.create(
                model=gpt_model,  
                messages=[
                    {"role": "system", "content": gpt_assistant_prompt},
                    {"role": "user", "content": generate_gpt_user_prompt(parameter_list, omission_probability)}],
                temperature=temperature_coefficient,
                max_tokens=500,
                frequency_penalty= frequency_penalty_coefficient,
                top_p = top_p_coefficient
            )

            # Printing result from call to GPT client
            print(completion.choices[0].message.content, '\n\n')

            
            # Now iterate and count the responses
            for line in completion.choices[0].message.content.split('\n'):
                
                match = re.match(r"\[(\w+), (.+?)\]", line)

                if match:
                    state_number, state_name = match.groups()
                else:
                    print(f"ERROR at iteration {iteration}: No match found")
                    error_counter +=1

                print(f'Appending: {state_number}, {state_name}')
                response_sheet["E"+str(iteration+2)] = state_number
                response_sheet["F"+str(iteration+2)] = state_name

                response_sheet["G"+str(iteration+2)] = 'NaN'

                response_sheet["H"+str(iteration+2)] = parameter_list[0]
                response_sheet["I"+str(iteration+2)] = parameter_list[1]
                response_sheet["J"+str(iteration+2)] = parameter_list[2]
                response_sheet["K"+str(iteration+2)] = parameter_list[3]
                response_sheet["L"+str(iteration+2)] = parameter_list[4]
                response_sheet["M"+str(iteration+2)] = parameter_list[5]
                response_sheet["N"+str(iteration+2)] = parameter_list[6]
                response_sheet["O"+str(iteration+2)] = parameter_list[7]

            print('\n')

        print(f'completed {iteration+1} iterations with {error_counter} match errors')
            
        response_book.save(workbook_path)




************************************
********* expression ID: LLM_WFI
************************************

~~~~~~~~~~~~ Iteration 00
[S01, Waiting for Input] 


Appending: S01, Waiting for Input


~~~~~~~~~~~~ Iteration 01
[S01, Waiting for Input] 


Appending: S01, Waiting for Input


~~~~~~~~~~~~ Iteration 02
[S01, Waiting for Input] 


Appending: S01, Waiting for Input


~~~~~~~~~~~~ Iteration 03
[S01, Waiting for Input] 


Appending: S01, Waiting for Input


~~~~~~~~~~~~ Iteration 04
[S01, Waiting for Input] 


Appending: S01, Waiting for Input


~~~~~~~~~~~~ Iteration 05
[S01, Waiting for Input] 


Appending: S01, Waiting for Input


~~~~~~~~~~~~ Iteration 06
[S02, Analyzing Object] 


Appending: S02, Analyzing Object


~~~~~~~~~~~~ Iteration 07
[S01, Waiting for Input] 


Appending: S01, Waiting for Input


~~~~~~~~~~~~ Iteration 08
[S05, Confused] 


Appending: S05, Confused


~~~~~~~~~~~~ Iteration 09
[S01, Waiting for Input] 


Appending: S01, Waiting for Input


~~~~~~~~~~~

### Single Call

In [28]:
# SETUP

parameter_list = [
        "left",
        "backward",
        "left",
        "low",
        "user",
        "short",
        "fast",
        "shaky"]

omission_probability = 0.5

In [29]:
completion = client.chat.completions.create(
    model=gpt_model,
    messages=[
        {"role": "system", "content": gpt_assistant_prompt},
        {"role": "user", "content": generate_gpt_user_prompt(parameter_list, omission_probability)}],
    temperature=temperature_coefficient,
    max_tokens=500,
    frequency_penalty= frequency_penalty_coefficient,
    top_p = top_p_coefficient
)

print('Raw Model Output:\n\n')
print(completion.choices[0].message)

Raw Model Output:


ChatCompletionMessage(content='[S05, Confused]', role='assistant', function_call=None, tool_calls=None, refusal=None)


In [30]:
print(f'''
      Parameter Config:\n
P01_value = {parameter_list[0]} Body Tilt
P02_value = {parameter_list[1]} Body Lean
P03_value = {parameter_list[2]} Body Turn
P04_value = {parameter_list[3]} Body Height
P05_value = {parameter_list[4]} Body Direction
P06_value = {parameter_list[5]} Pose Duration
P07_value = {parameter_list[6]} Motion Velocity
P08_value = {parameter_list[7]} Motion Smoothness
''')


print(completion.choices[0].message.content)


      Parameter Config:

P01_value = left Body Tilt
P02_value = backward Body Lean
P03_value = left Body Turn
P04_value = low Body Height
P05_value = user Body Direction
P06_value = short Pose Duration
P07_value = fast Motion Velocity
P08_value = shaky Motion Smoothness

[S05, Confused]
